# Technology Site Hacker News submissions: An analysis

In this project, the analysis revolves around a dataset of submissions to the popular technology news site, Hacker News. Hacker News is a platform where users can submit posts and engage in discussions related to technology, startups, and other topics. Posts on Hacker News can fall into various categories, with a particular focus on two types: "Ask HN" and "Show HN."

Ask HN posts are submissions where users seek answers or opinions from the Hacker News community, often in the form of questions or queries.
Show HN posts are submissions where users showcase something they have created, such as a project, product, or interesting discovery.
The primary objectives are:

* Determining if "Ask HN" or "Show HN" posts receive more comments on average.
* Identifying whether there's a specific time of day when posts tend to receive more comments on average.

To achieve this, analysis is conducted on a dataset that includes information about each post, such as its title, URL (if applicable), the number of points it received (calculated as upvotes minus downvotes), the number of comments, the author, and the submission date and time.

The next step involves loading the dataset and exploring its contents to gain insights and answer the aforementioned questions.

### Step 1: Data preparation
#### First, the csv file will be read in as a list of lists:
Source: https://www.kaggle.com/datasets/hacker-news/hacker-news-posts

(The used csv file is a shortened version of the source csv file)


In [63]:
from csv import reader

opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)

print(hn[:5])




[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


#### Now, the header gets separated from the rest of the rows to prepare the dataset for analysis:

In [64]:
headers = hn[0]
hn = hn[1:]
print(headers)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


#### Next, new lists are created that filter out posts starting with Ask HN or Show HN:

In [65]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title = title.lower()
    
    if title.startswith("ask hn"):
        ask_posts.append(row)
    elif title.startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)

print(len(ask_posts))
print(len(show_posts))
print(len(other_posts)) 

1744
1162
17194


### Step 2: Analysis
#### Comparing the average of comments received by Ask HN and Show HN posts:

In [66]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments

avg_ask_comments = total_ask_comments / len(ask_posts)
    
total_show_comments = 0

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
    
avg_show_comments = total_show_comments / len(show_posts)

print("Average ask posts comments:", int(avg_ask_comments))
print("Average show posts comments:", int(avg_show_comments))

Average ask posts comments: 14
Average show posts comments: 10


From the analysis, it is evident that "Ask HN" posts receive more comments on average than "Show HN" posts. Ask HN posts, where users seek answers or opinions from the Hacker News community, tend to generate more engagement and discussions, resulting in a higher average number of comments.

This insight can be valuable for users who are interested in maximizing engagement on their Hacker News submissions. If the goal is to generate discussions and receive feedback, creating "Ask HN" posts might be more effective. On the other hand, "Show HN" posts, which showcase projects or creations, may receive fewer comments on average but can still be valuable for sharing and visibility within the community.

### Analysis of Ask Posts and Comment Engagement by Hour
In this section, the focus shifts towards the exclusive analysis of "Ask HN" posts. The objective is to ascertain whether the time of day when these posts are created correlates with the number of comments they receive. To accomplish this, we will execute the following steps:

#### 1. Calculate Post Volume and Comments by Hour
The initial step involves the calculation of "Ask HN" post volume generated during each hour of the day. Additionally, we will tally the total count of comments garnered by these posts within each corresponding hour. 

In [67]:
import datetime as dt

result_list = []

for row in ask_posts:
    created_at = row[6]  
    num_comments = int(row[4])
    
    result_list.append([created_at, num_comments])

print("Excerpt of list with times of post creation and comments:\n", result_list[:5], "\n")

counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    date, time = row[0].split(" ")
    dt_date = dt.datetime.strptime(date, "%m/%d/%Y")
    dt_time = dt.datetime.strptime(time, "%H:%M")
    
    hour = dt.datetime.strftime(dt_time, "%H")
    num_comments = int(row[1])
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = num_comments
        
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += num_comments

print("Count of post creations by hour:\n", counts_by_hour, "\n")
print("Comments by hour:\n", comments_by_hour, "\n")

Excerpt of list with times of post creation and comments:
 [['8/16/2016 9:55', 6], ['11/22/2015 13:43', 29], ['5/2/2016 10:14', 1], ['8/2/2016 14:20', 3], ['10/15/2015 16:38', 17]] 

Count of post creations by hour:
 {'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58} 

Comments by hour:
 {'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641} 



#### 2: Compute the Average Comments per Hour

Building upon the data derived from step 1, the subsequent analysis will involve the calculation of the average number of comments per "Ask HN" post, segmented by the hour in which they were created. This process will enable the identification of potential patterns indicating specific hours of the day that tend to attract more comments.

Through this analysis, valuable insights will be provided regarding the optimal timing for posting "Ask HN" queries. Such insights will facilitate the maximization of engagement and the receipt of meaningful responses from the Hacker News community.

In [71]:
avg_by_hour = []
for hour in counts_by_hour:
    avg_by_hour.append([hour, comments_by_hour[hour]/counts_by_hour[hour]])

print("Average average number of comments for posts created during each hour of the day:\n", avg_by_hour)

Average average number of comments for posts created during each hour of the day:
 [['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['02', 23.810344827586206], ['18', 13.20183486238532], ['03', 7.796296296296297], ['05', 10.08695652173913], ['19', 10.8], ['01', 11.383333333333333], ['22', 6.746478873239437], ['08', 10.25], ['04', 7.170212765957447], ['00', 8.127272727272727], ['06', 9.022727272727273], ['07', 7.852941176470588], ['11', 11.051724137931034]]


Lastly, the top 5 hours for "Ask HN" post comments will be determined:

In [78]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print(swap_avg_by_hour, "\n")

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print("Top 5 Hours for Ask Posts Comments:")

for row in sorted_swap[:5]:
    average = row[0]
    hours = row[1]
    hours_dt = dt.datetime.strptime(hours, "%H")
    hours_converted = hours_dt.strftime("%H:%M")
    output = "{}: {:.2f} average comments per post".format(hours_converted, average)
    print(output)
    
    
    
    


[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']] 

Top 5 Hours for Ask Posts Comments:
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


#### Result: Optimal Hours for Higher Comment Engagement

Based on the analysis of "Ask HN" posts, the above hours have been identified as optimal for receiving higher comments on posts. The original findings were reported in Eastern Time Zone (US) and can be transformed to CET:

In [84]:
print("Top 5 Hours for Ask Posts Comments in CET:")

for row in sorted_swap[:5]:
    average = row[0]
    hours = row[1]
    
    hours_dt = dt.datetime.strptime(hours, "%H")
    hours_cet = hours_dt + dt.timedelta(hours=6)
    hours_converted = hours_cet.strftime("%H:%M")
    output = "{}: {:.2f} average comments per post".format(hours_converted, average)
    print(output)

Top 5 Hours for Ask Posts Comments in CET:
21:00: 38.59 average comments per post
08:00: 23.81 average comments per post
02:00: 21.52 average comments per post
22:00: 16.80 average comments per post
03:00: 16.01 average comments per post


These adjusted hours can serve as a reference for determining the optimal times to create "Ask HN" posts in Munich (CET) to maximize comment engagement based on the original findings from the dataset in US Eastern Time (EST).